In [1]:
from utils import *
import pandas as pd
import mne
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.preprocessing import normalize, StandardScaler
plt.style.use('seaborn-whitegrid')

In [2]:
def get_threshold(scores):
    scores = np.array(scores)
    lower_threshold = scores.mean() - (scores.std()/2)
    upper_threshold = scores.mean() + (scores.std()/2)
    return scores.mean(), (lower_threshold,upper_threshold)

def get_stress_type(score, grade):
    """ Non-stress (0): score < lower_threshold
        Neutral    (1): lower_threshold <= score <= upper_threshold
        Stress     (2): score > lower_threshold """
    if(score < grade[0]):
        return 0
    elif(score <= grade[1]):
        return 1
    elif(score > grade[1]):
        return 2

def PSS_printer(PSS):
    # peak at info
    temp = PSS.popitem()
    PSS[temp[0]] = temp[1]
    column = list(temp[1].keys())
    space = "\t\t"
    print(f"Name{space}",f"{space}".join(column),sep="" )
    print("="*60)
    for name, info in PSS.items():
        print(f"{name}{space}",sep="",end="")
        for col in column:
            print(f"{info[col]}{space}",end="")

        print()


TYPE_DEF = {0:'Non-Stress', 1:'Neutral', 2: 'Stress'}

In [3]:
PSS = dict()
scores = []
with open('./PSS_scores.csv','r') as f:
    f.readline() # skip header
    for line in f.readlines(): 
        name,score = line.split(',')
        PSS[name] = {'score':int(score)}
        scores.append(int(score))

mean, grade = get_threshold(scores)
# print(f"Total={len(PSS)} | Mean={mean} | Lower Thres={grade[0]} | Higher Thres={grade[1]}")

type_count = {0:0, 1:0, 2:0}
for name, dict_info in PSS.items():
    label = get_stress_type(dict_info['score'], grade)
    dict_info['type'] = label
    dict_info['type_definition'] = TYPE_DEF[label]
    type_count[label] = type_count[label] + 1

# print(f"Non Stress={type_count[0]} | Neutral={type_count[1]} | Stress={type_count[2]}")

# PSS_printer(PSS)

In [4]:
sampling_rate = 125 #Hz
files = glob(f"data/*.csv")
for f in tqdm(files):
    name = f.split('/')[1].split('__')[0]
    pd_raw = pd.read_csv(f, dtype={'Marker':str})
    pd_raw = pd_raw.drop(columns='timestamps')
    raw = dataframe_to_raw(pd_raw, sfreq=sampling_rate)
    PSS[name]['raw'] = raw
    # print(f"{name} | time: {len(pd_raw)/125}")

  0%|          | 0/55 [00:00<?, ?it/s]

In [5]:
# save(PSS,"PSS")

PSS = load("PSS")

In [5]:
for name, info in tqdm(PSS.items()):
    raw = info['raw']
    # raw.filter(l_freq=1,h_freq=None, method='iir', iir_params={'order':3.0, 'ftype':'butter'}, verbose=False) # Slow drift
    # raw.notch_filter(freqs=[50])
    # epochs = mne.Epochs(raw, np.array([[125*60*1, 0, 1]]), tmin=0, tmax=30, baseline=(0,30), verbose=False)
    # print(name)
    # a = epochs.plot_psd(picks=['F3','F4','T3','T4'])
    # print("="*40)

  0%|          | 0/55 [00:00<?, ?it/s]

In [5]:
def get_freq(PSS):
    # peak at info
    temp = PSS.popitem()
    PSS[temp[0]] = temp[1]
    raw = temp[1]['raw']
    power,freq = mne.time_frequency.psd_welch(raw,n_fft=125, verbose=True)
    return freq

freq = get_freq(PSS)
print(freq)

Effective window size : 1.000 (s)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62.]


In [6]:
band_names = np.array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta'])
filter_list = [[1,3],[4,7],[8,12],[13,30],[30,43], [4,13], [13,17]]
bands = []
for filt in filter_list:
    pt = np.argwhere((freq >= filt[0]) & (freq <= filt[1])).reshape(-1)
    bands.append(pt)
bands = np.array(bands)
print(bands)

[array([1, 2, 3]) array([4, 5, 6, 7]) array([ 8,  9, 10, 11, 12])
 array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
        30])
 array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])
 array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13])
 array([13, 14, 15, 16, 17])]


/tmp/ipykernel_5535/3256231302.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bands = np.array(bands)


In [7]:
def get_markers():
    sampling_rate = 125 #Hz
    # 15/60 = 0.25
    step_minutes = np.arange(0,5,0.25)
    print(f"{step_minutes=}")
    step_minutes = np.expand_dims(step_minutes * sampling_rate * 60, axis=1)
    markers = np.concatenate( [step_minutes, np.zeros( step_minutes.shape ), np.ones( step_minutes.shape ) ], axis=1  ).astype(np.int64)
    return markers
markers = get_markers()
# markers

step_minutes=array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 ,
       2.75, 3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75])


In [18]:
info['type_definition']

'Neutral'

In [10]:
# features = None
csv = None
names = []
stress_types = []
stress_defs = []
for name,info in PSS.items():
    if(info['type'] == 1): continue
    raw = info['raw']
    epochs = mne.Epochs(raw, markers, tmin=0, tmax=15, baseline=None, verbose=False)
    for evoked in epochs.iter_evoked():
        names.append(name)
        stress_types.append(info['type'])
        stress_defs.append(info['type_definition'])
        row = None
        feature = None
        slow, gamma = None, None
        a_f3, a_f4 = None, None
        a_t7, a_t8 = None, None
        b_f3, b_f4 = None, None
        b_t7, b_t8 = None, None
        for index, band in enumerate(bands):
            power,freq = mne.time_frequency.psd_welch(raw,n_fft=125, verbose=False)
            power = power.squeeze()
            power = 10 * np.log10(power)
            data = power[::,band].mean(axis=1).reshape(1,-1)
            # print(f"{data.shape=}")

            if(type(row) == type(None)): row = data.copy()
            else: row = np.concatenate([row,data.copy()], axis=1)

            # print(f"{row.shape=}")
            # for asym
            if(band_names[index] == 'Alpha'):
                a_f3 = data[:,raw.ch_names.index('F3')]
                a_f4 = data[:,raw.ch_names.index('F4')]
                # We use t3 as t7 and t4 as t8
                a_t7 = data[:,raw.ch_names.index('T3')]
                a_t8 = data[:,raw.ch_names.index('T4')]
            if(band_names[index] == 'Beta'):
                b_f3 = data[:,raw.ch_names.index('F3')]
                b_f4 = data[:,raw.ch_names.index('F4')]
                # We use t3 as t7 and t4 as t8
                b_t7 = data[:,raw.ch_names.index('T3')]
                b_t8 = data[:,raw.ch_names.index('T4')]

            ####### Mean for visualization #######
            data = data.mean().reshape(1,-1)
            # for relative gamma
            if(band_names[index] == 'Slow'): slow = data
            if(band_names[index] == 'Gamma'): gamma = data

            if(type(feature) == type(None)): feature = data
            else: feature = np.concatenate([feature, data], axis=1)
        # print(feature.shape)
        # the eighth feature: relative gamma is slow/gamma
        relative_gamma = slow/gamma
        feature = np.concatenate([feature, relative_gamma], axis=1)
        # The asymetry
        alpha_frontal = ((a_f4 - a_f3) / (a_f4 + a_f3)).reshape(1,-1)
        feature = np.concatenate([feature, alpha_frontal], axis=1)
        # alpha_temporal
        alpha_temporal = ((a_t8 - a_t7) / (a_t8 + a_t7)).reshape(1,-1)
        feature = np.concatenate([feature, alpha_temporal], axis=1)
        # alpha_asymmetry
        alpha_asymmetry = alpha_frontal + alpha_temporal
        feature = np.concatenate([feature, alpha_asymmetry], axis=1)
        # beta_frontal
        beta_frontal = ((b_f4 - b_f3) / (b_f4 + b_f3)).reshape(1,-1)
        feature = np.concatenate([feature, beta_frontal], axis=1)
        # beta_temporal
        beta_temporal = ((b_t8 - b_t7) / (b_t8 + b_t7)).reshape(1,-1)
        feature = np.concatenate([feature, beta_temporal], axis=1)

        row = np.concatenate([row, relative_gamma, alpha_frontal, alpha_temporal, alpha_asymmetry, beta_frontal, beta_temporal], axis=1)
    # print(slow/gamma)
    # print(feature.shape)
    # print(feature)
        info['feature'] = feature
        if(type(csv) == type(None)): csv = row
        else: csv = np.concatenate([csv,row], axis=0)
    print(f"{csv.shape=}")
    # break
    # if(type(features) == type(None)): features = feature
    # else: features = np.concatenate([features, feature], axis=0)
# print(features.shape)


csv.shape=(20, 118)
csv.shape=(40, 118)
csv.shape=(60, 118)
csv.shape=(80, 118)
csv.shape=(100, 118)
csv.shape=(120, 118)
csv.shape=(140, 118)
csv.shape=(160, 118)
csv.shape=(180, 118)
csv.shape=(200, 118)
csv.shape=(220, 118)
csv.shape=(240, 118)
csv.shape=(260, 118)
csv.shape=(280, 118)
csv.shape=(300, 118)
csv.shape=(320, 118)
csv.shape=(340, 118)
csv.shape=(360, 118)
csv.shape=(380, 118)
csv.shape=(400, 118)
csv.shape=(420, 118)
csv.shape=(440, 118)
csv.shape=(460, 118)
csv.shape=(480, 118)
csv.shape=(500, 118)
csv.shape=(520, 118)
csv.shape=(540, 118)
csv.shape=(560, 118)
csv.shape=(580, 118)
csv.shape=(600, 118)
csv.shape=(620, 118)
csv.shape=(640, 118)
csv.shape=(660, 118)
csv.shape=(680, 118)
csv.shape=(700, 118)
csv.shape=(720, 118)


In [18]:
with open("feature_15s_head.csv","w") as f:
    # print(raw.ch_names[:-1])
    # print(band_names)
    header = ["name"]
    header.append('type')
    for band_name in band_names:
        for ch_name in raw.ch_names[:-1]:
            header.append(f"{ch_name}_{band_name}")
    header.append("relative_gamma")
    header.append("alpha_frontal")
    header.append("alpha_temporal")
    header.append("alpha_asymmetry")
    header.append("beta_frontal")
    header.append("beta_temporal")
    f.write(",".join(header))


In [12]:
np.savetxt("feature_15s.csv", csv, delimiter=",")

In [15]:
print(stress_types)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [14]:
print(names)

['amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'amp', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'aui', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bam', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bank', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'bas', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beau', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer', 'beer',

In [19]:
a = pd.read_csv("feature_15s.csv")
a

,name,type,Fp1_Delta,Fp2_Delta,F3_Delta,F4_Delta,F7_Delta,F8_Delta,C3_Delta,C4_Delta,...,P3_Low_beta,P4_Low_beta,O1_Low_beta,O2_Low_beta,relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal
0,amp,0,-107.929198,-106.278321,-110.729355,-107.924797,-107.256658,-107.325092,-113.636412,-114.788803,...,-120.764314,-122.586631,-117.978331,-118.428572,0.899633,0.005791,0.005519,0.011310,0.015338,0.007906
1,amp,0,-107.929198,-106.278321,-110.729355,-107.924797,-107.256658,-107.325092,-113.636412,-114.788803,...,-120.764314,-122.586631,-117.978331,-118.428572,0.899633,0.005791,0.005519,0.011310,0.015338,0.007906
2,amp,0,-107.929198,-106.278321,-110.729355,-107.924797,-107.256658,-107.325092,-113.636412,-114.788803,...,-120.764314,-122.586631,-117.978331,-118.428572,0.899633,0.005791,0.005519,0.011310,0.015338,0.007906
3,amp,0,-107.929198,-106.278321,-110.729355,-107.924797,-107.256658,-107.325092,-113.636412,-114.788803,...,-120.764314,-122.586631,-117.978331,-118.428572,0.899633,0.005791,0.005519,0.011310,0.015338,0.007906
4,amp,0,-107.929198,-106.278321,-110.729355,-107.924797,-107.256658,-107.325092,-113.636412,-114.788803,...,-120.764314,-122.586631,-117.978331,-118.428572,0.899633,0.005791,0.005519,0.011310,0.015338,0.007906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,yong,2,-100.732702,-100.497997,-111.217229,-108.980626,-96.052525,-97.390635,-106.698244,-102.448917,...,-117.016443,-115.856541,-115.459331,-113.060211,0.873401,-0.000330,-0.010663,-0.010993,-0.001452,-0.012932
716,yong,2,-100.732702,-100.497997,-111.217229,-108.980626,-96.052525,-97.390635,-106.698244,-102.448917,...,-117.016443,-115.856541,-115.459331,-113.060211,0.873401,-0.000330,-0.010663,-0.010993,-0.001452,-0.012932
717,yong,2,-100.732702,-100.497997,-111.217229,-108.980626,-96.052525,-97.390635,-106.698244,-102.448917,...,-117.016443,-115.856541,-115.459331,-113.060211,0.873401,-0.000330,-0.010663,-0.010993,-0.001452,-0.012932
718,yong,2,-100.732702,-100.497997,-111.217229,-108.980626,-96.052525,-97.390635,-106.698244,-102.448917,...,-117.016443,-115.856541,-115.459331,-113.060211,0.873401,-0.000330,-0.010663,-0.010993,-0.001452,-0.012932


In [81]:
b = a.T[:10].T.copy()

In [84]:
b.to_csv("feature_less.csv")